In [1]:
import os 
import pandas as pd

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI

from langchain_chroma import Chroma
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

In [2]:
from dotenv import load_dotenv
# Load environment variables
load_dotenv('../../.env')

True

In [3]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0, max_tokens = 1000)

In [4]:
df_articles = pd.read_csv('../../data/df_articles.csv').dropna()
df_articles['content'] = df_articles['title'] + df_articles['content']

df_articles_analytics = pd.read_csv('../../data/df_articles_analytics.csv').dropna()
df_articles_analytics['content'] = df_articles_analytics['title'] + df_articles_analytics['content']

documents = df_articles['content'].to_list() + df_articles_analytics['content'].to_list()
with open('../../data/courses.txt', 'r') as f:
    documents += list(f.read().split('\n\n\n\n'))

In [5]:
import os
import PyPDF2

def load_pdfs_from_folder(folder_path="../../data/pdf/"):
    documents = []

    for filename in os.listdir(folder_path):
        if filename.lower().endswith(".pdf"):
            file_path = os.path.join(folder_path, filename)

            with open(file_path, "rb") as f:
                pdf_reader = PyPDF2.PdfReader(f)
                text = ""
                for page in pdf_reader.pages:
                    page_text = page.extract_text()
                    if page_text:
                        text += page_text + " "

            documents.append(text)

    return documents

In [6]:
pdf_docs = load_pdfs_from_folder()

In [7]:
len(pdf_docs)

17

In [9]:
documents += pdf_docs

In [11]:
documents = [doc.replace('\xa0', ' ').replace('\n', ' ') for doc in documents]

In [12]:
len(documents)

559

In [13]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_text(" ".join(documents))

In [14]:
len(chunks)

37152

In [15]:
# create the open-source embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="DeepPavlov/rubert-base-cased-sentence", model_kwargs={'device': 'cpu'})

db = Chroma.from_texts(chunks, embedding_function, persist_directory="../../artifacts/chroma_db")

/var/folders/nf/9qms21p955z2mzc061kwnfs80000gn/T/ipykernel_19451/3267848779.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embedding_function = SentenceTransformerEmbeddings(model_name="DeepPavlov/rubert-base-cased-sentence", model_kwargs={'device': 'cpu'})
No sentence-transformers model found with name DeepPavlov/rubert-base-cased-sentence. Creating a new one with mean pooling.


In [16]:
import pickle


with open("../../artifacts/embedding_function.pkl", "wb") as file:
    pickle.dump(embedding_function, file)

In [17]:
retriever = db.as_retriever()

In [18]:
from langchain.prompts import ChatPromptTemplate

# Multi Query: Different Perspectives
template = """You are an AI language model assistant. Your task is to generate five 
different versions of the given user question to retrieve relevant documents from a vector 
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search. 
Provide these alternative questions separated by newlines. Original question: {question}"""
prompt_perspectives = ChatPromptTemplate.from_template(template)

from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

generate_queries = (
    prompt_perspectives 
    | llm
    | StrOutputParser() 
    | (lambda x: x.split("\n"))
)

In [19]:
from langchain.load import dumps, loads, load

def get_unique_union(documents: list[list]):
    """ Unique union of retrieved docs """
    # Flatten list of lists, and convert each Document to string
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    # Get unique documents
    unique_docs = list(set(flattened_docs))
    # Return
    return [loads(doc) for doc in unique_docs]

In [20]:
question = "Какие акции IT компаний сейчас наиболее интересны для приобретения?"
retrieval_chain = generate_queries | retriever.map() | get_unique_union
docs = retrieval_chain.invoke({"question":question})
len(docs)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/var/folders/nf/9qms21p955z2mzc061kwnfs80000gn/T/ipykernel_19451/3273285394.py:10: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  return [loads(doc) for doc in unique_docs]


12

In [21]:
docs

[Document(page_content='Мы не исключаем, что такая стратегия развития продолжится в дальнейшем и поможет группе увеличить спектр предлагаемых продуктов и услуг. Компания планирует добиться этого за счет: Менеджмент Группы Аренадаты прогнозирует рост числа заказчиков в шесть раз в среднесрочной перспективе. Для достижения этой цели компания планирует расширить партнерскую сеть и отдел прямых продаж вдвое. Отдельное внимание получит сегмент малого и среднего бизнеса, для которого актуальны облачные разработки. Что с финансами компании Основную выручку компании приносят продажи лицензий. Группа Аренадата также зарабатывает на реализации услуг техобслуживания, консалтинге и внедрении ПО. Небольшая часть доходов приходится на продажу подписок на ПО и услуг по обучению. Компания предлагает три вида лицензий. Постоянная лицензия Предусматривает ограниченное количество инсталляций на неограниченный срок. Неограниченная'),
 Document(page_content='бизнес — электронная коммерция. Это говорит о по

In [22]:
from operator import itemgetter

# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    {"context": retrieval_chain, 
     "question": itemgetter("question")} 
    | prompt
    | llm
    | StrOutputParser()
)

res = final_rag_chain.invoke({"question":question})

print(res)

На основании предоставленного контекста, можно выделить несколько факторов, которые делают акции IT-компаний интересными для приобретения:

1. **Развитие собственных продуктов**: Увеличение доли собственных продуктов в общем объеме продаж, что положительно сказывается на рентабельности. Например, в первом полугодии 2024 года доля собственных продуктов выросла до 30% с менее 3% в 2020 году.

2. **Рост валовой маржи**: Валовая прибыль от собственных продуктов составляет около 60%, в то время как от сторонних — всего около 13%. Это указывает на более высокую рентабельность собственных решений.

3. **Консолидация рынка**: Активные слияния и поглощения (M&A) в IT-секторе, которые могут привести к расширению бизнеса и увеличению рыночной доли.

4. **Импортозамещение**: Позитивное влияние на прибыльность компаний в условиях активного импортозамещения.

Таким образом, акции IT-компаний, которые активно развивают собственные продукты и участвуют в M&A-сделках, могут быть наиболее интересными дл

In [107]:
from langchain_core.load.serializable import to_json_not_implemented

repr = to_json_not_implemented(final_rag_chain)

In [108]:
repr

{'lc': 1,
 'type': 'not_implemented',
 'id': ['langchain_core', 'runnables', 'base', 'RunnableSequence'],
 'repr': "{\n  context: ChatPromptTemplate(input_variables=['question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='You are an AI language model assistant. Your task is to generate five \\ndifferent versions of the given user question to retrieve relevant documents from a vector \\ndatabase. By generating multiple perspectives on the user question, your goal is to help\\nthe user overcome some of the limitations of the distance-based similarity search. \\nProvide these alternative questions separated by newlines. Original question: {question}'))])\n           | ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x308f7a490>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x3424bcb90>, root_client=<openai.OpenAI object at 0x177459690>, root_async_client=<openai.AsyncOpenAI obj

In [104]:
string_representation = dumps(final_rag_chain, pretty=True)
print(string_representation[:500])

{
  "lc": 1,
  "type": "not_implemented",
  "id": [
    "langchain_core",
    "vectorstores",
    "base",
    "VectorStoreRetriever"
  ],
  "repr": "VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x111af9090>)",
  "name": "VectorStoreRetriever"
}


In [109]:
import json

with open("../../artifacts/chain.json", "w") as fp:
    json.dump(repr, fp)

In [120]:
with open("../../artifacts/chain.json", "r") as fp:
    chain_dict = json.load(fp)

In [121]:
chain_dict

{'lc': 1,
 'type': 'not_implemented',
 'id': ['langchain_core', 'runnables', 'base', 'RunnableSequence'],
 'repr': "{\n  context: ChatPromptTemplate(input_variables=['question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='You are an AI language model assistant. Your task is to generate five \\ndifferent versions of the given user question to retrieve relevant documents from a vector \\ndatabase. By generating multiple perspectives on the user question, your goal is to help\\nthe user overcome some of the limitations of the distance-based similarity search. \\nProvide these alternative questions separated by newlines. Original question: {question}'))])\n           | ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x308f7a490>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x3424bcb90>, root_client=<openai.OpenAI object at 0x177459690>, root_async_client=<openai.AsyncOpenAI obj